In [ ]:
# coding: utf-8
import os
import sys
from glob import glob
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import mne
from mne.decoding import UnsupervisedSpatialFilter
from sklearn.decomposition import PCA, FastICA, FactorAnalysis, IncrementalPCA
from sklearn.externals import joblib

sys.path.append('src')
import eegf
import functions

sns.set_style('whitegrid',
             {'xtick.bottom': True,
              'ytick.left': True})
mpl.rcParams['font.size'] = 14
mpl.rcParams['axes.titlesize'] = 'medium'
mpl.rcParams['axes.labelsize'] = 'medium'
mpl.rcParams['xtick.labelsize'] = 'medium'
mpl.rcParams['ytick.labelsize'] = 'medium'
mpl.rcParams['legend.fontsize'] = 'medium'
mpl.rcParams['figure.titlesize'] = 'medium'
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.edgecolor'] = 'k'
million = 1000000
mpl.rcParams['font.size'] = 20

eegf.mkdir('data/pca_rotation/')

def topomap(w, info, axes=None, show=False):
    a = np.abs(w).max()
    return mne.viz.plot_topomap(w, info, axes=axes, show=show)

# Isolating Preperatory Motor Activity

Traditionally, the Readiness Potential is recorded in the seconds prior to self-initiated actions.
This means that no stimuli were presented immediately prior to the action in question,
and as a result the EEG does not contain stimulus-evoked activity.
In the current experiment, each trial begins with the a gamble being shown onscreen,
and responses typically occured approximately 1 s after the gamble was presented.
As a result, the EEG prior to the action is contaminated by components evoked by the stimuli.

Therefore, it is necessary to preprocess the data in order to isolate 
the EEG components related to motor preparation.
To do this, we adapt a procedure introduced by 
[Kayser and Tencke (2006)](https://doi.org/10.1016/j.clinph.2005.08.034):
first apply a surface Laplacian filter to the data to help localise EEG components,
then use Principle Components Analysis with varimax rotation to separate out components,
and finally use topographical and temporal properties to identify each component.

In this notebook, I work with a random subset of our experimental data.
This procedure requires two sets of epochs from the raw EEG:
stimulus-locked epochs, capturing activity locked to the onset of the stimulus on each trial, and
response-locked epochs, capturing motor preperation and execution.

## Stimulus-locked epochs

In [ ]:
# stim_epochs[::10].save('CSD_PCA/data/stim_epo.fif')

In [ ]:
## Load data
stim_epochs = mne.read_epochs('data/trial_epo.fif').apply_baseline((-.1, 0)).crop(-.5, 3)
# stim_epochs = stim_epochs[::2] # Drop every second trial
## Drop outliers
stim_rej = eegf.find_outlier_trials(stim_epochs, thresh=120./million)
stim_epochs = stim_epochs[stim_rej==False]

In [ ]:
%mkdir -p figures/sup/

In [ ]:
mpl.rcParams['font.size'] = 20

In [ ]:
## Plot channels
eegf.plot_joint(stim_epochs.average(), 
                times=[.125, .2, .27, .5, .625],
                title='Stimulus-locked epochs - raw data');
plt.savefig('figures/sup/raw_stim.svg')

## Response-locked epochs

In [ ]:
response_epochs = mne.read_epochs('data/response_epo.fif').apply_baseline((-3., -2.9)).crop(-3, .5)
# response_epochs = response_epochs[::2] # Drop every second trial
resp_rej = eegf.find_outlier_trials(response_epochs, thresh=120./million)
response_epochs = response_epochs[resp_rej==False]

In [ ]:
# response_epochs[::10].save('CSD_PCA/data/resp_epo.fif')

In [ ]:
## Plot channels
eegf.plot_joint(response_epochs.average(), 
                times=[-1, -.05, 0],
                title='Stimulus-locked epochs - raw data');
plt.savefig('figures/sup/raw_resp.svg')

As you can see, the data is dominated by the P300 (or centro-parietal positivity; CPP) response that occurs approxamitely 500 ms after stimulus onset.
In the response-locked data, we can see that most channels have a positive slope
in the moments prior to action, with a small negative spike around channel FCz just at the time of action.
This reflects the fact that our EEG contains a relatively small motor preperation component,
superimposed over a massive stimulus-evoked component.
We can see the same thing looking at just data from FCz (where the motor component is maximal)
and Pz (where the CPP is maximal).



In [ ]:
chan_indices = dict(zip(stim_epochs.ch_names, range(32)))

plt.figure(figsize=(20, 5))
for i, epochs, title in zip([1,2], 
                            [stim_epochs, response_epochs],
                            ['Stimulus-locked', 'Response-locked']):
    X = epochs.get_data()[:, :32] * million # Trial x Channel x Time
    plt.subplot(1, 2, i)
    for ch in ['Fcz', 'Pz']:
        ch_i = chan_indices[ch]
        eegf.plot_mean_sem(X[:, ch_i], epochs.times, label=ch)
    plt.title(title)
    plt.legend()
    eegf.flipy()
    plt.xlabel('Time (s)')
    plt.ylabel(u'μV')
plt.show()    

# Surface Laplacian

Our first step is to apply a surface Laplacian filter to the data to help localise EEG components.
This transforms the data from raw voltages, measured in μV,
to estimates of the Current Source Density (CSD), measured in $\frac{μV^2}{mm}$.
In practice, this means that we're spatially filtering the data to attenuate signals
with a low spatial frequency - that is, those that cover a large region of the scalp.
We do this in order to prevent the CPP signal from contaminating other electrodes far from Pz.

In [ ]:
## Apply surface laplacian filter
stim_epochs_csd = eegf.surface_laplacian(stim_epochs, m=5).apply_baseline((-.1, 0))
## Plot channels
eegf.plot_joint(stim_epochs_csd.average(), 
                times=[.125, .2, .27, .5, .625],
                title='Stimulus-locked epochs - CSD');
plt.savefig('figures/sup/csd_stim.svg')

In the filtered epochs, we can still see the CPP response,
but it is restricted to a much narrower region of the scalp.
Note that the signal is no longer measured in μV, so we cannot compare the magnitude of the component
before and after filtering.
With the CPP restricted to around Pz, we can now see a second, negative component centered around FCz.

Next, we apply the same filter to the response-locked data.
This allows us to the the ramping negative component at FCz - an analogue to the RP.

In [ ]:
## Filter
response_epochs_csd = eegf.surface_laplacian(response_epochs, m=5)
## Plot
eegf.plot_joint(response_epochs_csd.average(), 
                times=[-1, -.05, 0],
                title='Response-locked epochs - CSD');
plt.savefig('figures/sup/csd_resp.svg')

In [ ]:
response_epochs = mne.read_epochs('data/response_epo.fif')
response_epochs_csd = eegf.surface_laplacian(response_epochs, m=5)

In [ ]:
resp_rej = eegf.find_outlier_trials(response_epochs, thresh=120./million)
response_epochs = response_epochs[resp_rej==False]
response_epochs_csd = response_epochs_csd[resp_rej==False]

In [ ]:
# response_epochs.get_data()[0,0] * million

In [ ]:
plt.figure(figsize=(20, 5))
for i, epochs, title in zip([1,2], 
                            [stim_epochs_csd, response_epochs_csd],
                            ['Stimulus-locked', 'Response-locked']):
    X = epochs.get_data()[:, :32] * million # Trial x Channel x Time
    plt.subplot(1, 2, i)
    for ch in ['Fcz', 'Pz']:
        ch_i = chan_indices[ch]
        eegf.plot_mean_sem(X[:, ch_i], epochs.times, label=ch)
    plt.title(title)
    plt.legend()
    eegf.flipy()
    plt.xlabel('Time (s)')
plt.suptitle('Current Source Density')
plt.show()    

# Tuning the Surface Laplacian
 
Note that the surface Laplacian function takes an additional argument, `m`,
which controlls the smoothness of the filter. 
In the code above, I've set `m=5`, which produced good results for this dataset. 

Lower values of `m` produce more spikey, locally isolated patterns of activity.
Higher values produce smoother activity across broad areas of the scalp.

In [ ]:
# %%capture --no-display
# for m in [3,4,5,6,10]:
#     E = eegf.surface_laplacian(response_epochs, m=m)
#     eegf.plot_joint(E.average(), 
#                     times=[-1, -.05, 0],
#                     title='Response-locked epochs - CSD (m = %i)' % m)
#     plt.show()

I'll return to this issue later.


# Principle Components

The next step is to use PCA to isolate neural components in the data.
We're most interested in finding the premotor component that peaks just before action,
so we perform PCA on the 1 second of data leading up to the response on each trial.



In [ ]:
X = response_epochs_csd.copy().crop(-.2, 0).get_data()[:, :32] # Trials x Channels x Time

The first step here is to calculate the covariance matrix for the 32 channels.
There are a number of ways to do this on epoched EEG data,
but the most straightforward way here is to calculate the covariance matrix for each trial,
and then average across trials.
I've found almost no difference when using other methods to do PCA.
Mike X Cohen's book ([code available here](http://www.mikexcohen.com/#books)]
is a good introduction to this and other methods.


In [ ]:
cov_by_trial = np.array([np.cov(X[i] - X[i].mean()) 
                         for i in range(X.shape[0])])
cov = cov_by_trial.mean(0)
plt.figure(figsize=(8,6))
sns.heatmap(cov, cmap='seismic', center=0);
plt.savefig('figures/sup/cov.svg')

In [ ]:
cov

Next, we use numpy's `eig` function to extract the eigenvalues and eigvectors from this covariance matrix.

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(cov)
## Sort largest first
ix = np.flip(np.argsort(eig_vals))
eig_vals = eig_vals[ix]
eig_vecs = eig_vecs[:, ix]


In [ ]:
eig_vals, eig_vecs = np.linalg.eig(cov)
ix = np.flip(np.argsort(eig_vals))
eig_vals = eig_vals[ix]
eig_vecs = eig_vecs[:, ix] # Is this the right axis? - Yes

# plt.plot(np.log(eig_vals[:5] / eig_vals.mean()), '-o')
# plt.plot(eig_vals[:12] / eig_vals.mean(), '-o')
# plt.hlines(1, linestyle='dashed', *plt.xlim())

# Eigenvalues
print (eig_vals[:4] / eig_vals.mean())

## Variance explained
ve = eig_vals / eig_vals.sum()
print( ve[:4])

## Residual variance explained
for i in range(4):
    print (ve[i] / (1-ve[:i].sum()))


## Eigenvectors

Each of the 32 eigenvectors indicates how much each of
the original 32 channels weights onto each of the 32 PCA components.
Every column in the plot below corresponds to an eigenvector.

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(eig_vecs, cmap='seismic', center=0);
plt.xlabel('Eigenvector')
plt.ylabel('Loading onto electrode')
plt.yticks(range(32), epochs.ch_names, size=6, rotation=.45)
# plt.savefig('figures/sup/eigvecs.svg')
plt.show()

Or, scaling the eigenvectors by how much variance they explain (the eigenvalues)

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(eig_vecs * eig_vals, cmap='seismic', center=0)
plt.xlabel('Eigenvector')
plt.ylabel('Loading onto electrode')
plt.yticks(range(32), epochs.ch_names, size=6, rotation=.45)
plt.savefig('figures/sup/eigvecs2.svg')
plt.show()

We can better see these weights by plotting them according to where each channel is on the scalp.

In [ ]:
plt.figure(figsize=(14, 8))
for i in range(32):
    ax = plt.subplot(4, 8, i+1)
    topomap(eig_vecs[:, i], response_epochs.info, axes=ax)
    plt.title('Component %i' % (i+1))
plt.show()

## Eigenvalues

Eigenvalues reflect the variance of each PCA component. 
By normalising them to sum to 1, we can see how much of the variance in the original data 
(the second prior to action)
is explained by each component.

In [ ]:
variance_explained = eig_vals / eig_vals.sum()
plt.plot(range(1, len(variance_explained)+1), variance_explained*100, '-o')
plt.hlines(variance_explained.mean()*100, linestyle='dashed', *plt.xlim())
plt.ylabel('% variance explained')
plt.xlabel('Component')
plt.xticks(range(0, 33, 4))
plt.savefig('figures/sup/var_explained.svg')
plt.xlim(0, 33)

The dotted line shows the average of all 32 eigenvalues.
An alternative is to divide the eigvenvalues by their mean,
yielding standardised eigenvalues.

In [ ]:
standard_eigvals = eig_vals / eig_vals.mean()
plt.plot(range(1, len(standard_eigvals)+1), standard_eigvals, '-o')
plt.hlines(1, linestyle='dashed', *plt.xlim())
plt.ylabel('Standardised eigenvalue')
plt.xlabel('Component')
plt.xticks(range(0, 33, 4))
plt.savefig('figures/sup/eigenvalues.svg')
plt.xlim(0, 33)

Clearly, some of the components are more important than others.
For the next step, we want to keep only the components that reflect important neural activity,
and throw away those that contain only noise.
There is no one right way to decide how many components to keep,
but the standard rules of thumb are either to keep everything with a stanardised eigenvalue greater than 1
(everything above the dashed line in the plot above),
or to identify the "elbow" at which the plot of the eigvenvalues levels out, 
so that including extra components after the elbow will not explain much more variance.

In the plot above, there are elbows at component 4, and at 9.
In practice, I've found that our results don't change much for any number of components between 4 and 9,
and component 9 has the last standardised eigenvalue above 1.
Therefore, I decide to retain components 1-9 here.

In [ ]:
# mpl.rcParams['font.size'] = 10


In [ ]:
n_to_retain = 9

plt.figure(figsize=(14, 2))
for i in range(n_to_retain):
    ax = plt.subplot(1, n_to_retain, i+1)
    topomap(eig_vecs[:, i], response_epochs.info, axes=ax)
    plt.title('C%i' % (i+1))
plt.savefig('figures/sup/pca_components.svg')
plt.show()

# PCA Rotation

Now that we have selected a subset of PCA components of interest,
we can transform the EEG data from it's original representation (32 channels)
to it's PCA represenation (in this case, 9 components).
We do this by multiplying the raw data by the eigvector for each component.

In [ ]:
def rotate_eeg(X, L):
    '''
    X: EEG array (trials x channels x times)
    L: Rotation matrix (original channels x rotated components)
    '''
    return np.stack([X[i].T.dot(L) for i in range(X.shape[0])], axis=0).swapaxes(2, 1)

pca_rot_mat = eig_vals[:n_to_retain] * eig_vecs[:, :n_to_retain] # 32 x 9
## Get raw data
stimX = stim_epochs_csd.get_data()[:, :32] * million # Units are somewhat arbitrary
respX = response_epochs_csd.get_data()[:, :32] * million
## Rotate data
stimX_pca = rotate_eeg(stimX, pca_rot_mat)
respX_pca = rotate_eeg(respX, pca_rot_mat)

In [ ]:
plt.figure(figsize=(12, 5))
for comp in range(n_to_retain):
    eegf.plot_mean_sem(stimX_pca[:, comp], stim_epochs.times, label='Component %i' % (comp+1))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Stimulus-locked PCA Components')
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
for comp in range(n_to_retain):
    eegf.plot_mean_sem(respX_pca[:, comp], response_epochs.times, label='Component %i' % (comp+1))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Response-locked PCA Components')
plt.show()

In [ ]:
# from functions import pca_component_plot
%mkdir -p figures/sup/pca_eeg/
for i in range(9):
    fig = pca_component_plot(i, pca_rot_mat, stimX_pca, respX_pca, stim_epochs, response_epochs)    
    if i < 8:
        for j in [1, 2]:
            ax = fig.axes[j]
            ax.set_xlabel('')
            ax.set_xticklabels([])
    plt.savefig('figures/sup/pca_eeg/c%i.svg' % (i+1))
    plt.show()

# Varimax Rotation

However, these components don't look quite right.
PCA works by first extracting out the largest component, then the largest component from what's left, 
repeating this untill all components have been found.
This means that component 1 is greedy, and tries to explain as much variance as possible.
This is a problem for neural components, as we would expect
the brain to produce several components of approximately the same size.
To address this, we use what's called *varimax rotation*.
Varimax attempts to rotate the PCA components we have found
so the the variance is better shared across components
(of course, this is a simplification, but it's enough for our purposes).

The plot above shows our 9 components after varimax rotation.

In [ ]:
pca_rot_mat = eig_vals[:n_to_retain] * eig_vecs[:, :n_to_retain] # 32 x 9
varimax_rot_mat = eegf.varimax(pca_rot_mat, method='varimax').T

plt.figure(figsize=(14, 2))
for i in range(n_to_retain):
    ax = plt.subplot(1, n_to_retain, i+1)
    topomap(varimax_rot_mat[:, i], response_epochs.info, axes=ax)
    plt.title('C%i' % (i+1))
plt.show()

In [ ]:
def rotate_eeg(X, L):
    '''
    X: EEG array (trials x channels x times)
    L: Rotation matrix (original channels x rotated components)
    '''
    return np.stack([X[i].T.dot(L) for i in range(X.shape[0])], axis=0).swapaxes(2, 1)

In [ ]:
## Rotate data
stimX_varimax = rotate_eeg(stimX, varimax_rot_mat)
respX_varimax = rotate_eeg(respX, varimax_rot_mat)

In [ ]:
plt.figure(figsize=(12, 5))
for comp in range(n_to_retain):
    eegf.plot_mean_sem(stimX_varimax[:, comp], stim_epochs.times, label='Component %i' % (comp+1))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Stimulus-locked Varimax Components')
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
for comp in range(n_to_retain):
    eegf.plot_mean_sem(respX_varimax[:, comp], response_epochs.times, label='Component %i' % (comp+1))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Response-locked Varimax Components')
plt.show()

For both PCA and varimax components, the sign of each componet is arbirary -
we might find a positive component, that decreases over time, or a negative component that increases.
I find it useful to flip the signs of these components so that each one is increasing 
in the buildup to action.


In [ ]:
t_baseline, t_action = response_epochs.time_as_index([-2, 0])
respXmean = respX_varimax.mean(0) # Average across trials
respXmean_slope = respXmean[:, t_action] - respXmean[:, t_baseline] # Positive if component increases
sign_flip = np.where(respXmean_slope > 0, 1, -1)

for i, s in enumerate(sign_flip):
    varimax_rot_mat[:, i] *= s
stimX_varimax = rotate_eeg(stimX, varimax_rot_mat)
respX_varimax = rotate_eeg(respX, varimax_rot_mat)

In [ ]:
plt.figure(figsize=(14, 2))
for i in range(n_to_retain):
    ax = plt.subplot(1, n_to_retain, i+1)
    topomap(varimax_rot_mat[:, i], response_epochs.info, axes=ax)
    plt.title('C%i' % (i+1))
plt.savefig('figures/sup/vmax_components.svg')
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
for comp in range(n_to_retain):
    eegf.plot_mean_sem(stimX_varimax[:, comp], stim_epochs.times, label='Component %i' % (comp+1))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Stimulus-locked Varimax Components')
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
for comp in range(n_to_retain):
    eegf.plot_mean_sem(respX_varimax[:, comp], response_epochs.times, label='Component %i' % (comp+1))
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Response-locked Varimax Components')
plt.show()

In [ ]:
# from functions import pca_component_plot
    
%mkdir -p figures/sup/vmax_eeg/
for i in range(9):
    fig = pca_component_plot(i, varimax_rot_mat, stimX_varimax, respX_varimax, stim_epochs, response_epochs)    
    if i < 8:
        for j in [1, 2]:
            ax = fig.axes[j]
            ax.set_xlabel('')
            ax.set_xticklabels([])
    plt.savefig('figures/sup/vmax_eeg/c%i.svg' % (i+1))
    plt.show()

Finally, let's create a new mne.Epochs object to store the rotated data, copy the metadata across to it, and save it. 

In [ ]:
## Create info object
info1 = mne.create_info(n_to_retain, sfreq=response_epochs.info['sfreq'], ch_types='eeg')
stim_epochs_vmax = mne.EpochsArray(stimX_varimax[:, :n_to_retain], 
                                   info1, tmin=stim_epochs.times[0],
                                   baseline=(-.1, 0))
stim_epochs_vmax.metadata = stim_epochs.metadata
stim_epochs_vmax.save('data/stim_vmax_epo.fif')

info2 = mne.create_info(n_to_retain, sfreq=response_epochs.info['sfreq'], ch_types='eeg')
resp_epochs_vmax = mne.EpochsArray(respX_varimax[:, :n_to_retain],
                                   info2, tmin=response_epochs.times[0],
                                   baseline=(-3, -2.9))
resp_epochs_vmax.metadata = response_epochs.metadata
resp_epochs_vmax.save('data/resp_vmax_epo.fif')

We can now use these epochs as we would any other data in MNE,
except that we need to be aware that

- the epochs don't contain any information about "channel" locations, and
- the signal is no longer measured in μV (although the plots will still show this by default)

In [ ]:
E = resp_epochs_vmax[::5]
rt_order = np.argsort(E.metadata['rt'])
fig = plt.figure(figsize=(12, 8))
g = mne.viz.plot_epochs_image(E, 1, ## Channels start at 0
                              vmin=-.25, vmax=.25,
                              order=rt_order, overlay_times=-1*E.metadata['rt'],
                              ts_args=dict(show_sensors=False), # Don't show location of 'sensor'
                             fig=fig);

In [ ]:
E = stim_epochs_vmax[::5]
rt_order = np.argsort(E.metadata['rt'])
fig = plt.figure(figsize=(12, 8))
g = mne.viz.plot_epochs_image(E, 1, ## Channels start at 0
                              vmin=-.25, vmax=.25,
                              order=rt_order, overlay_times=1*E.metadata['rt'],
                              ts_args=dict(show_sensors=False), # Don't show location of 'sensor'
                             fig=fig);